In [74]:
def dlt_homography(I1pts, I2pts):
    """
    Find perspective Homography between two images.

    Given 4 points from 2 separate images, compute the perspective homography
    (warp) between these points using the DLT algorithm.

    Parameters:
    ----------- 
    I1pts  - 2x4 np.array of points from Image 1 (each column is x, y).
    I2pts  - 2x4 np.array of points from Image 2 (in 1-to-1 correspondence).

    Returns:
    --------
    H  - 3x3 np.array of perspective homography (matrix map) between image coordinates.
    A  - 8x9 np.array of DLT matrix used to determine homography.
    """
    #--- FILL ME IN ---
    A = np.zeros([8,9])    #create the A matrix, an 8x9 np array
    
    for i,point in enumerate(I1pts.T):   #iterate over the vector of points in Image 1, transposed
        SecondImage = I2pts.T[i]           #find point in second image
        x = point[0]       #x coord 1st pic
        y = point[1]       # y coord 1st pic
        x_2 = SecondImage[0]          #x coord 2nd pic
        y_2 = SecondImage[1]           #y coord 2nd pic
        A_i = np.array([ [-1*x, -1*y, -1, 0 , 0 ,0, x_2*x, x_2*y, x_2],[0 ,0, 0, -1*x, -1*y, -1, y_2*x, y_2*y, y_2],])   
        # from formula in thesis paper ith element of A
        A[i*2 : (i+1)*2,:] = A_i   #finalize A matrix
    
    #create and reshape H matrix to a 3x3 np array
    H = null_space(A)
    H = H.reshape(3,3)
    last = H[2][2]            
    H = H * (1/last)        #normalize H 
    #------------------

    return H, A


def bilinear_interp(I, pt):
    """
    Performs bilinear interpolation for a given image point.

    Given the (x, y) location of a point in an input image, use the surrounding
    4 pixels to conmpute the bilinearly-interpolated output pixel intensity.

    Note that images are (usually) integer-valued functions (in 2D), therefore
    the intensity value you return must be an integer (use round()).

    This function is for a *single* image band only - for RGB images, you will 
    need to call the function once for each colour channel.

    Parameters:
    -----------
    I   - Single-band (greyscale) intensity image, 8-bit np.array (i.e., uint8).
    pt  - 2x1 np.array of point in input image (x, y), with subpixel precision.

    Returns:
    --------
    b  - Interpolated brightness or intensity value (whole number >= 0).
    """
    #--- FILL ME IN ---
    
    if pt.shape != (2, 1):
        raise ValueError('Point size is incorrect.')
    
    #obtain the surrounding 4 pixels
    x_1 = int(pt[0]) - 1
    x_2 = int(pt[0]) + 1
    y_1 = int(pt[1]) - 1
    y_2 = int(pt[1]) + 1

    # we will use the polynomial fit method to determine the bilinear interpolation
    A = np.array([[1, x_1, y_1, x_1 * y_1],[1, x_1, y_2, x_1 * y_2],[1, x_2, y_1, x_2 * y_1],[1, x_2, y_2, x_2 * y_2]])
    A_inv = inv(A)
    
    Q = np.array([[I[y_1, x_1]],[I[y_2, x_1]],[I[y_1, x_2]], [I[y_2, x_2]]])

    x = A_inv @ Q

    
    b = int(x[0] + x[1] * pt[0] + x[2] * pt[1] + x[3] * pt[0]* pt[1])    #compute polynomial expression for intensity
    b = round(b)    #round b

    #------------------

    return b

def histogram_eq(I):
    """
    Histogram equalization for greyscale image.

    Perform histogram equalization on the 8-bit greyscale intensity image I
    to produce a contrast-enhanced image J. Full details of the algorithm are
    provided in the Szeliski text.

    Parameters:
    -----------
    I  - Single-band (greyscale) intensity image, 8-bit np.array (i.e., uint8).

    Returns:
    --------
    J  - Contrast-enhanced greyscale intensity image, 8-bit np.array (i.e., uint8).
    """
    #--- FILL ME IN ---
    if I.dtype != np.uint8:   # Verify I is grayscale.
        raise ValueError('Incorrect image format!')
    
    shaped = I.shape
    flat_image = I.flatten() #flatten image
    H, bins = np.histogram(flat_image, bins = 255, range = (0,255))
    C = H.cumsum() #cumulative density function
    last=C[-1]
    C = 255 * C / last #normalize CDF
    # Linear interpolation of CDF to find new pixel values
    J = np.interp(flat_image, bins[:-1], C)
            
    J = J.reshape(shaped)
    #-------------------

    return J

import numpy as np
from matplotlib.path import Path
from imageio import imread, imwrite

#from dlt_homography import dlt_homography
#from bilinear_interp import bilinear_interp
#from histogram_eq import histogram_eq

def billboard_hack():
    """
    Hack and replace the billboard!

    Parameters:
    ----------- 

    Returns:
    --------
    Ihack  - Hacked RGB intensity image, 8-bit np.array (i.e., uint8).
    """
    # Bounding box in Y & D Square image.
    bbox = np.array([[404, 490, 404, 490], [38,  38, 354, 354]])

    # Point correspondences.
    Iyd_pts = np.array([[416, 485, 488, 410], [40,  61, 353, 349]])
    Ist_pts = np.array([[2, 218, 218, 2], [2, 2, 409, 409]])

    Iyd = imread('../billboard/yonge_dundas_square.jpg')
    Ist = imread('../billboard/uoft_soldiers_tower_dark.png')

    Ihack = np.asarray(Iyd)
    Ist = np.asarray(Ist)

    #--- FILL ME IN ---
    #--------
    # Let's do the histogram equalization first.
    Ist_eq = histogram_eq(Ist) #equalize soldiers tower

    # Compute the perspective homography we need...
    H, A = dlt_homography(Iyd_pts, Ist_pts)

    path = Path(Iyd_pts.T) #compute poly path
    # Main 'for' loop to do the warp and insertion - 
    # this could be vectorized to be faster if needed!
    # You may wish to make use of the contains_points() method
    # available in the matplotlib.path.Path class!
    min_1 = min(bbox[0])
    max_1 = max(bbox[0])        #process bounds of bounding box
    min_2 = min(bbox[1])
    max_2 = max(bbox[1])
    for i in range (min_1, max_1+1):          #points of bounding box, iterate over range of bounding box
            for j in range (min_2, max_2+1):   
                    if path.contains_points([[i, j]]):    #use contains_points(), is points in polygon path?
                            X = np.array ([[i],[j], [1]])
                            Ih_stower = H @ X  #matrix multiply H with X  
                            Ih_stower = Ih_stower/Ih_stower[-1]   # normalize homogenized soldiers tower image
                            Ihack[j][i] = bilinear_interp(Ist_eq, Ih_stower[:-1, :]) #perform bilinear interpolation to produce hacked image
    #------------------

    # plt.imshow(Ihack)
    # plt.show()
    imwrite('billboard_hacked.png',Ihack);

    return Ihack





In [76]:
tar cvf handin.tar*.py


SyntaxError: invalid syntax (<ipython-input-76-aa172d286134>, line 1)

In [75]:
billboard_hack()

array([[[242, 243, 245],
        [242, 243, 245],
        [242, 243, 245],
        ...,
        [250, 250, 252],
        [250, 250, 252],
        [250, 250, 252]],

       [[242, 243, 245],
        [242, 243, 245],
        [242, 243, 245],
        ...,
        [250, 250, 252],
        [250, 250, 252],
        [250, 250, 252]],

       [[242, 243, 245],
        [242, 243, 245],
        [242, 243, 245],
        ...,
        [250, 250, 252],
        [250, 250, 252],
        [250, 250, 252]],

       ...,

       [[119, 118, 116],
        [115, 114, 112],
        [117, 116, 114],
        ...,
        [135, 137, 136],
        [143, 145, 144],
        [153, 155, 154]],

       [[129, 128, 126],
        [133, 132, 130],
        [129, 128, 126],
        ...,
        [140, 142, 141],
        [141, 143, 142],
        [149, 151, 150]],

       [[115, 115, 115],
        [113, 113, 113],
        [134, 134, 134],
        ...,
        [145, 147, 146],
        [124, 126, 125],
        [131, 133, 132]]

In [52]:
Ihack = billboard_hack()
imwrite(Ihack, 'billboard_hacked.png');


ValueError: Image is not numeric, but str.